In [2]:
'''
Checks if guess is the same as answer
gives back an array, 
0 : wrong letter
1 : right letter, wrong spot
2 : right letter, right spot
'''
def checkWord(guess, ans):
    correct = [0, 0, 0, 0, 0]
    used = [' ', ' ', ' ', ' ', ' ']
    for i in range(len(ans)):
        if (guess[i] == ans[i]):
            correct[i] = 2
            used[i] = guess[i]
    
    for i in range(len(ans)):
        if (correct[i] == 2):
            continue
        if (ans.count(guess[i]) > 0 and used.count(guess[i]) == 0):
            correct[i] = 1
    
    return correct

def getandcheckWord(ans, wordlist : list):
    while(1):
        print("Enter a five letter word")
        guess = input()
        if (len(guess) == 5 and wordlist.count(guess) > 0):
            break
        print("Not a valid word")
    print(f"Your guess is: {guess}")
    return checkWord(guess, ans)



In [21]:
def GetWordList():
    f = open("words.txt", "r")
    wordlist = []
    for word in f:
        word = word[:-1]
        wordlist.append(word)
    f.close()
    return wordlist

def GetAllowedWordList():
    f = open("allowedwords.txt", "r")
    wordlist = []
    for word in f:
        word = word[:-1]
        wordlist.append(word)
    f.close()
    return wordlist

def game(WordList: list):
    guesses = 0
    gameOver = False
    answer = GetRandomWord(WordList)
    print(f"The answer is {answer}")
    for i in range(20):
        print(WordList[i])
        print(ExpectedValue(WordList[i], WordList))
        print()
    while (guesses < 6):
        correct = getandcheckWord(answer)
        print(correct)
        if (correct.count(2) == 5):
            gameOver = True
            break
        guesses += 1

    if (gameOver):
        print("u win")
    else:
        print("game over, u lost")

    print(f"The answer was {answer}")

from random import randint 
def GetRandomWord(WordList: list):
    i = randint(0,len(WordList))
    return WordList[i]

In [30]:

# Step 1: Go through every word, calculate the expected value of how many words will be left after this guess
def PruneWordList(guess : str, wordlist : list, guessResult : list) -> list:
    deletedWords = []
    for word in wordlist:
        # Check if the word fits the guess string
        tmpguess = list(word)
        delete = False

        for i in range(5):
            if guessResult[i] == 2:
                if word[i] != guess[i]:
                    delete = True
                    break
                else:
                    tmpguess[i] = '*' # tmp char as we have already used this letter to satisfy a requirement
        
        for i in range(5):
            if guessResult[i] == 1:
                placed = False
                for j in range(5):
                    if i == j and word[i] == guess[i]:
                        delete = True
                        break   
                    elif tmpguess[j] == guess[i]:
                        tmpguess[j] = '*' # tmp char as we have used this positionally
                        placed = True
                        break
                if not placed:
                    delete = True

        for i in range(5):
            if guessResult[i] == 0:
                for j in range(5):
                    if word[i] == guess[i]:
                        delete = True
                        break
        if delete:
            deletedWords.append(word)
            continue
    for word in deletedWords:
        wordlist.remove(word)
    return deletedWords
            
                
        



# Step 2: 
def ExpectedValue(word: str, wordlist: list) -> int:
    # we return the expected number of words that will be left after this guess (lower is better)

    guessResult = [0,0,0,0,0]
    res = []
    for i in range(243):
        # if i == 111:
        #     print(len(wordlist))
        deletedWords = PruneWordList(word, wordlist, guessResult)
        res.append(len(deletedWords))

        # if i == 111:
        #     print("Test")
        #     print(guessResult)

        # put words back in 
        wordlist.extend(deletedWords)

        # assert(len(wordlist) > 12800 and len(wordlist) < 14000)
        # update correctness array
        idx = 4
        guessResult[idx] += 1
        while guessResult[idx] == 3:
            guessResult[idx] = 0
            idx -= 1
            guessResult[idx] += 1
    # print(res)
    
    return sum(res)


def FindBestWord(wordlist: list):

    wordpairs = []
    cnt = 0
    for word in wordlist:
        wordpairs.append([ExpectedValue(word, wordlist), word])
        cnt +=1
        if cnt == 20:
            break
    wordpairs.sort()
    f = open("answer.txt", "w")
    for wordpair in wordpairs:
        f.write(f"{wordpair[0]} {wordpair[1]}\n")
    f.close()


In [31]:
# words = GetWordList()
words = GetAllowedWordList()
FindBestWord(words)




In [ ]:
# sum : wordsleft / totalwords * totalwords - wordleft / totalwords

# probability we get a guessResult * the percentage of words left after we get this result